**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [4]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
!pip install scikit-video
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Dropout



# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

This function returns the next action to do following the $\epsilon$-greedy policy.

The agent chooses a random action with propability $\epsilon$ (exploration) or the action that maximizes the $Q$ function in state $s$ instead (exploitation) instead with propability $1-\epsilon$.

So there is a trade-off between exploration and exploitation when setting $\epsilon$.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=70 # set small when debugging
epochs_test=30 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array position is used to store the position of the agent on the map during the game, (x,y)=1 if the agent is on (x,y) or 0 other wise. <br>
The array board stores the rewards signals the agent gets whent it raaches a location. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        action = np.random.randint(0, self.n_action)
        return action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset()
        game_over = False
        
        win = 0
        lose = 0

        while game_over == False:
            # Do an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win += reward
            if reward < 0:
                lose -= reward

            # Apply the reinforcement strategy
            #loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [11]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.0/14.0. Average score (-5.0)
Win/lose count 7.5/11.0. Average score (-4.25)
Win/lose count 8.5/4.0. Average score (-1.3333333333333333)
Win/lose count 7.0/5.0. Average score (-0.5)
Win/lose count 9.0/14.0. Average score (-1.4)
Win/lose count 10.0/13.0. Average score (-1.6666666666666667)
Win/lose count 11.0/22.0. Average score (-3.0)
Win/lose count 12.0/16.0. Average score (-3.125)
Win/lose count 14.0/19.0. Average score (-3.3333333333333335)
Win/lose count 12.0/16.0. Average score (-3.4)
Win/lose count 11.0/13.0. Average score (-3.272727272727273)
Win/lose count 13.5/10.0. Average score (-2.7083333333333335)
Win/lose count 11.5/21.0. Average score (-3.230769230769231)
Win/lose count 7.5/16.0. Average score (-3.607142857142857)
Win/lose count 12.5/11.0. Average score (-3.2666666666666666)
Win/lose count 7.0/7.0. Average score (-3.0625)
Win/lose count 11.0/12.0. Average score (-2.9411764705882355)
Win/lose count 8.0/13.0. Average score (-3.0555555555555554)
Win/lose cou

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Question 5.1__


By definition : $$ Q^{\pi}(s,a) = E_{p^{\pi}}[\sum\limits_{t = 0}^{\infty}\gamma^{t}r(s_{t},a_{t})\|s_0=s,a_0=a]$$ 


$$  Q^{\pi}(s,a) = E_{p^{\pi}}[r(s,a) + \sum\limits_{t = 1}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_0=s,a_0=a]$$ 


$$  Q^{\pi}(s,a) = E_{p^{\pi}}[r(s,a) + \gamma E_{p^{\pi}}[\sum\limits_{t = 1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_1=s',a_1=a']|s_0=s,a_0=a]$$ 

$$ = E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$$ 

__Question 5.2__ By applying the first equlity to $\pi$ we have:

$$Q^{\pi}(s,a)=E_{(s',a')\sim p^{\pi^{*}}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$$

$$Q^{\pi}(s,a)= r(s,a) + \gamma\sum\limits_{s'\in S}p(s'|s,a)\pi(a'|s')Q^{\pi}(s',a')$$ and since $\pi$ follows the greedy policy we have: $\pi(a'|s') = 1$  function for $a' = argmax_{a'} Q^{\pi^{*}}(s',a')$ so the action-value function q can be rewritten as:

$$Q^{\pi}(s,a)= E_{s'\sim \pi(.|s,a)}[r(s,a)+\gamma max_{a'}Q^{\pi}(s',a')]$$

But since $\pi*$ is optimal, a greedy policy of that policy is the optimal policy itself, hence $\pi = \pi^{*}$

Finally, $$Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].$$


We have then proved that knowing the optimal action-value fucntion is equivalent to know the optimal policy since any greedy policy w.r.t to that value function is optimal.
 
__Question 5.3__
We want to decrease this gap between the prediction of our model $Q(s,a;\theta)$ and the target $E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]$ by adjusting the model parameters $\theta$. It results that a plausible objective to minimize is the loss 
$$\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$$



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)<self.max_memory:
            self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape((1,5,5,self.n_state))))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            
            input_states[i] = s_
            
            if game_over_:
                ######## FILL IN
                target_q[i,a_] = r_
            else:
                ######## FILL IN
                q_est = self.model.predict(np.expand_dims(n_s_, 0))
                target_q[i, a_] = r_ + self.discount * np.max(q_est)
                
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()

        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(16,activation ='relu')) 
        #model.add(Dense(64,activation ='relu')) 
        #model.add(Dense(128,activation ='relu')) 
        #model.add(Dense(256,activation ='relu')) 
        model.add(Dense(4))
        adadelta = keras.optimizers.Adadelta()
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.08, memory_size=2000, batch_size = 16)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))













Epoch 000/070 | Loss 0.0613 | Win/lose count 1.5/9.0 (-7.5)
Epoch 001/070 | Loss 0.0010 | Win/lose count 1.5/4.0 (-2.5)
Epoch 002/070 | Loss 0.0028 | Win/lose count 2.5/8.0 (-5.5)
Epoch 003/070 | Loss 0.0117 | Win/lose count 5.0/5.0 (0.0)
Epoch 004/070 | Loss 0.0074 | Win/lose count 1.5/3.0 (-1.5)
Epoch 005/070 | Loss 0.0052 | Win/lose count 3.5/2.0 (1.5)
Epoch 006/070 | Loss 0.0129 | Win/lose count 7.0/6.0 (1.0)
Epoch 007/070 | Loss 0.0046 | Win/lose count 4.5/3.0 (1.5)
Epoch 008/070 | Loss 0.0033 | Win/lose count 4.0/2.0 (2.0)
Epoch 009/070 | Loss 0.0114 | Win/lose count 7.0/2.0 (5.0)
Epoch 010/070 | Loss 0.0038 | Win/lose count 2.5/2.0 (0.5)
Epoch 011/070 | Loss 0.0222 | Win/lose count 2.5/4.0 (-1.5)
Epoch 012/070 | Loss 0.0047 | Win/lose count 1.5/3.0 (-1.5)
Epoch 013/070 | Loss 0.0139 | Win/lose count 3.5/1.0 (2.5)
Epoch 014/070 | Loss 0.0011 | Win/lose count 1.0/3.0 (-2.0)
Epoch 015/070 | Loss 0.0212 | Win/lose count 1.5/5.0 (-3.5)
Epoch 016/070 | Loss 0.0148 | Win/lo

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(64,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [17]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 16)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/070 | Loss 0.0063 | Win/lose count 2.5/6.0 (-3.5)
Epoch 001/070 | Loss 0.0007 | Win/lose count 6.0/4.0 (2.0)
Epoch 002/070 | Loss 0.0034 | Win/lose count 6.5/8.0 (-1.5)
Epoch 003/070 | Loss 0.0029 | Win/lose count 3.5/2.0 (1.5)
Epoch 004/070 | Loss 0.0054 | Win/lose count 3.0/4.0 (-1.0)
Epoch 005/070 | Loss 0.0023 | Win/lose count 4.5/3.0 (1.5)
Epoch 006/070 | Loss 0.0013 | Win/lose count 7.0/3.0 (4.0)
Epoch 007/070 | Loss 0.0018 | Win/lose count 11.0/2.0 (9.0)
Epoch 008/070 | Loss 0.1173 | Win/lose count 6.0/2.0 (4.0)
Epoch 009/070 | Loss 0.0060 | Win/lose count 13.5/6.0 (7.5)
Epoch 010/070 | Loss 0.0022 | Win/lose count 7.5/0 (7.5)
Epoch 011/070 | Loss 0.0004 | Win/lose count 10.0/3.0 (7.0)
Epoch 012/070 | Loss 0.0046 | Win/lose count 9.0/4.0 (5.0)
Epoch 013/070 | Loss 0.0024 | Win/lose count 11.5/3.0 (8.5)
Epoch 014/070 | Loss 0.0016 | Win/lose count 11.5/0 (11.5)
Epoch 015/070 | Loss 0.0012 | Win/lose count 14.5/3.0 (11.5)
Epoch 016/070 | Loss 0.0006 | Win/lose count 3.5/

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [18]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 16)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')

Test of the CNN
Win/lose count 11.0/3.0. Average score (8.0)
Win/lose count 7.5/0. Average score (7.75)
Win/lose count 10.5/2.0. Average score (8.0)
Win/lose count 4.0/2.0. Average score (6.5)
Win/lose count 8.5/4.0. Average score (6.1)
Win/lose count 3.5/3.0. Average score (5.166666666666667)
Win/lose count 10.0/2.0. Average score (5.571428571428571)
Win/lose count 6.0/0. Average score (5.625)
Win/lose count 4.5/2.0. Average score (5.277777777777778)
Win/lose count 10.5/1.0. Average score (5.7)
Win/lose count 11.0/2.0. Average score (6.0)
Win/lose count 9.5/3.0. Average score (6.041666666666667)
Win/lose count 10.5/3.0. Average score (6.153846153846154)
Win/lose count 13.0/4.0. Average score (6.357142857142857)
Win/lose count 19.5/3.0. Average score (7.033333333333333)
Win/lose count 4.5/3.0. Average score (6.6875)
Win/lose count 13.5/2.0. Average score (6.970588235294118)
Win/lose count 2.5/0. Average score (6.722222222222222)
Win/lose count 10.5/3.0. Average score (6.763157894736842

In [19]:
agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 16)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the FC
Win/lose count 8.0/4.0. Average score (4.0)
Win/lose count 2.5/3.0. Average score (1.75)
Win/lose count 3.5/3.0. Average score (1.3333333333333333)
Win/lose count 4.5/4.0. Average score (1.125)
Win/lose count 10.0/5.0. Average score (1.9)
Win/lose count 7.0/3.0. Average score (2.25)
Win/lose count 2.0/3.0. Average score (1.7857142857142858)
Win/lose count 5.0/7.0. Average score (1.3125)
Win/lose count 1.0/0. Average score (1.2777777777777777)
Win/lose count 2.0/2.0. Average score (1.15)
Win/lose count 2.5/4.0. Average score (0.9090909090909091)
Win/lose count 3.5/2.0. Average score (0.9583333333333334)
Win/lose count 6.0/2.0. Average score (1.1923076923076923)
Win/lose count 2.5/1.0. Average score (1.2142857142857142)
Win/lose count 1.5/4.0. Average score (0.9666666666666667)
Win/lose count 6.5/3.0. Average score (1.125)
Win/lose count 2.0/7.0. Average score (0.7647058823529411)
Win/lose count 8.0/3.0. Average score (1.0)
Win/lose count 3.0/0. Average score (1.1052631578

In [20]:
HTML(display_videos('cnn_test10.mp4'))

In [21]:
HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,epsilon=0.1,decay=0.95 ,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            agent.epsilon *= decay

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [0]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=T, temperature=0.3): #max_time=500
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        
        #Malus
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1
        
        reward = reward + self.board[self.x, self.y] 
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [46]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 16 ,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/070 | Loss 0.0126 | Win/lose count 9.5/54.20000000000004 (-44.70000000000004)
Epoch 001/070 | Loss 0.0112 | Win/lose count 16.0/30.100000000000037 (-14.100000000000037)
Epoch 002/070 | Loss 0.0291 | Win/lose count 20.0/18.599999999999994 (1.4000000000000057)
Epoch 003/070 | Loss 0.0155 | Win/lose count 20.5/17.299999999999994 (3.2000000000000064)
Epoch 004/070 | Loss 0.0148 | Win/lose count 21.0/15.799999999999981 (5.200000000000019)
Epoch 005/070 | Loss 0.0160 | Win/lose count 20.0/18.799999999999997 (1.2000000000000028)
Epoch 006/070 | Loss 0.0149 | Win/lose count 19.5/22.899999999999995 (-3.399999999999995)
Epoch 007/070 | Loss 0.0076 | Win/lose count 13.0/29.599999999999987 (-16.599999999999987)
Epoch 008/070 | Loss 0.0476 | Win/lose count 23.0/14.899999999999984 (8.100000000000016)
Epoch 009/070 | Loss 0.0322 | Win/lose count 23.5/14.599999999999989 (8.900000000000011)
Epoch 010/070 | Loss 0.0074 | Win/lose count 19.5/17.299999999999972 (2.2000000000000277)
Epoch 011/070

In [47]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 22.5/5.0. Average score (17.5)
Win/lose count 32.0/1.0. Average score (24.25)
Win/lose count 31.5/4.0. Average score (25.333333333333332)
Win/lose count 29.0/1.0. Average score (26.0)
Win/lose count 17.5/1.0. Average score (24.1)
Win/lose count 23.0/3.0. Average score (23.416666666666668)
Win/lose count 21.5/3.0. Average score (22.714285714285715)
Win/lose count 22.5/5.0. Average score (22.0625)
Win/lose count 22.0/1.0. Average score (21.944444444444443)
Win/lose count 20.0/3.0. Average score (21.45)
Win/lose count 23.5/3.0. Average score (21.363636363636363)
Win/lose count 27.0/1.0. Average score (21.75)
Win/lose count 19.5/2.0. Average score (21.423076923076923)
Win/lose count 27.0/2.0. Average score (21.678571428571427)
Win/lose count 24.5/4.0. Average score (21.6)
Win/lose count 24.5/0. Average score (21.78125)
Win/lose count 20.5/2.0. Average score (21.58823529411765)
Win/lose count 23.5/1.0. Average score (21.63888888888889)
Win/lose count 23.0/1.0. Average score (

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***

In [0]:
class Mimic_DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(Mimic_DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.95
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape((1,5,5,self.n_state))))
            

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_[:,:,1:], a_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):

            s_, a_  = self.memory.random_access()
            target_q[i, a_] = 5
            input_states[i] = s_

        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class Mimic_DQN_CNN(Mimic_DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(Mimic_DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(32,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(64,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), 'mse')
        #model.compile(adam(lr=lr, decay=1e-4), 'mse')
        self.model = model


In [0]:
def train_mimic_agent(expert,agent,env,epoch=70,epsilon=0.1, decay=0.95, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(epsilon)

        while not game_over:
            # The agent performs an action
            action = expert.act(state, train=False)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [63]:
expert = agent
mimic_agent = Mimic_DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 16, n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimic_agent(expert,mimic_agent, env, prefix='mimic_dqn_cnn')

Epoch 000/070 | Loss 2.4665 | Win/lose count 23.5/21.900000000000002 (1.5999999999999979)
Epoch 001/070 | Loss 3.7090 | Win/lose count 27.5/12.099999999999984 (15.400000000000016)
Epoch 002/070 | Loss 1.7878 | Win/lose count 17.0/15.899999999999984 (1.1000000000000156)
Epoch 003/070 | Loss 2.0650 | Win/lose count 28.0/13.799999999999981 (14.200000000000019)
Epoch 004/070 | Loss 1.1878 | Win/lose count 26.5/16.19999999999998 (10.300000000000018)
Epoch 005/070 | Loss 2.4070 | Win/lose count 21.5/14.299999999999981 (7.200000000000019)
Epoch 006/070 | Loss 2.9208 | Win/lose count 24.0/11.299999999999986 (12.700000000000014)
Epoch 007/070 | Loss 2.0643 | Win/lose count 25.0/17.799999999999986 (7.2000000000000135)
Epoch 008/070 | Loss 2.6488 | Win/lose count 23.5/18.399999999999988 (5.100000000000012)
Epoch 009/070 | Loss 2.2136 | Win/lose count 22.5/17.499999999999982 (5.000000000000018)
Epoch 010/070 | Loss 2.8081 | Win/lose count 23.5/16.79999999999999 (6.70000000000001)
Epoch 011/070 | L

In [66]:
# Evaluation of the Mimic Agent
env_test = Environment(grid_size=size, max_time=T,temperature=0.3)
print('Test of the Mimic Agent')
test(mimic_agent,env_test,epochs_test,prefix='cnn_mimic_agent_test')

Test of the Mimic Agent
Win/lose count 9.5/2.0. Average score (7.5)
Win/lose count 5.0/5.0. Average score (3.75)
Win/lose count 9.5/2.0. Average score (5.0)
Win/lose count 14.5/5.0. Average score (6.125)
Win/lose count 14.5/13.0. Average score (5.2)
Win/lose count 11.0/5.0. Average score (5.333333333333333)
Win/lose count 16.5/11.0. Average score (5.357142857142857)
Win/lose count 13.0/5.0. Average score (5.6875)
Win/lose count 12.0/1.0. Average score (6.277777777777778)
Win/lose count 6.5/3.0. Average score (6.0)
Win/lose count 6.0/4.0. Average score (5.636363636363637)
Win/lose count 12.0/10.0. Average score (5.333333333333333)
Win/lose count 15.5/5.0. Average score (5.730769230769231)
Win/lose count 3.0/3.0. Average score (5.321428571428571)
Win/lose count 11.0/1.0. Average score (5.633333333333334)
Win/lose count 13.0/4.0. Average score (5.84375)
Win/lose count 14.0/9.0. Average score (5.794117647058823)
Win/lose count 3.5/4.0. Average score (5.444444444444445)
Win/lose count 8.0/5

It  doesn't bring better results but it computs way faster.